In [1]:
using Revise

In [10]:
using Optim
using ForwardDiff
using Plots; gr()
using ColorTypes
using ReverseDiff
using Base.Test

In [3]:
import Nets
import LearningMPC

In [4]:
target = x -> [sin(x[1])]
# target = x -> x

(::#1) (generic function with 1 method)

In [5]:
training_pts = linspace(-2π, π, 5)
training_data = [
    ([x], hcat(target(x), ForwardDiff.jacobian(target, [x]))) 
    for x in training_pts];

In [34]:
struct CompiledGradient{Tape <: ReverseDiff.CompiledTape}
    tape::Tape
end

function (c::CompiledGradient)(∇, x)
    ReverseDiff.gradient!((∇,), c.tape, (x,))
end

In [104]:
function trainable_net(training_data)
    net = Nets.Net(zeros(Nets.Params{Float64}, [1, 10, 10, 1]), Nets.elu)
    for I in eachindex(net.params.data)
        net.params.data[I] += 0.1 * randn()
    end

    f = (params::AbstractVector) -> begin
        n = Nets.similar(net, params)
        q = [1.0 1.0]
        sum(training_data) do xyJ
            x, yJ = xyJ
            sum(abs2, q .* (Nets.predict_sensitivity(n, x) .- yJ))
        end
    end

    loss_tape = ReverseDiff.compile(ReverseDiff.GradientTape(f, (net.params.data,)))
    gradient_result = (similar(net.params.data),);

    g! = CompiledGradient(loss_tape)
    net, f, g!, loss_tape
end

net, f, g! = trainable_net(training_data)
@inferred f(net.params.data)
@inferred g!(similar(net.params.data), net.params.data)

([0.0469058, -0.00755694, -0.0274922, -0.00143651, -0.00644097, 0.0181396, -0.00341134, -0.00525958, -0.0120157, 0.0257308  …  -0.0102819, 0.0289733, -0.00214394, -0.00249579, -0.0630438, 0.0117527, -0.0338932, 0.0595366, 0.00252548, -0.559983],)

In [105]:
@show f(net.params.data)
solver = LBFGS()
# options = Optim.Options(iterations=10)
options = Optim.Options()
g_calls = 0
# @time @profile for i in 1:100
    results = optimize(f, g!, copy(net.params.data), solver, options)
    net.params.data .= results.minimizer;
    g_calls += results.g_calls
# end
@show f(net.params.data)
results

f(net.params.data) = 4.980463453226413
f(net.params.data) = 2.6393062155931597e-19


Results of Optimization Algorithm
 * Algorithm: L-BFGS
 * Starting Point: [-0.0023835034757604904,-0.11917370440714084, ...]
 * Minimizer: [-0.5532667683527774,-0.2669450325302178, ...]
 * Minimum: 2.639306e-19
 * Iterations: 162
 * Convergence: true
   * |x - x'| < 1.0e-32: false 
     |x - x'| = 1.93e-07 
   * |f(x) - f(x')| / |f(x)| < 1.0e-32: false
     |f(x) - f(x')| / |f(x)| = 9.65e-01 
   * |g(x)| < 1.0e-08: true 
     |g(x)| = 5.03e-09 
   * stopped by an increasing objective: false
   * Reached Maximum Number of Iterations: false
 * Objective Calls: 506
 * Gradient Calls: 506

In [106]:
g_calls

506

In [107]:
plt = plot([x[1] for (x, yJ) in training_data], 
    [yJ[1] for (x, yJ) in training_data],
    line=nothing,
    marker=:dot)
for (x, yJ) in training_data
    y = yJ[1, 1]
    slope = yJ[1, 2]
    θ = atan(slope)
    δx = 0.1 * cos(θ)
    xs = [x[1] - δx, x[1] + δx]
    ys = [y - slope * δx, y + slope * δx]
    plot!(plt, xs, ys, color=colorant"red", legend=nothing)
end

x_samples = linspace(-2π, 2π, 101)
plot!(plt, x_samples, [net([x])[1] for x in x_samples], color=colorant"blue")

plt

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 -5.0 
 
 
 -2.5 
 
 
 0.0 
 
 
 2.5 
 
 
 5.0 
 
 
 -6 
 
 
 -4 
 
 
 -2 
 
 
 0 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
<polyline clip-path="url(#clip2902)" style="stroke:#0000ff; stroke-width:1; stroke-opacity:1; fill:none" points="
 47.764,74.7456 52.9335,69.1375 58.1031,63.9243 63.2726,59.0944 68.4422,54.6401 73.6117,50.558 78.7813,46.8486 83.9508,43.5172 89.1204,40.5737 94.2899,38.033 
 99.4595,35.9153 104.629,34.2465 109.799,33.0643 114.968,32.4239 120.138,32.3722 125.307,32.9629 130.477,34.2574 135.646,36.326 140.816,39.2473 145.985,43.1099 
 151.155,48.0114 156.325,53.9673 161.494,60.0782 166.664,66.0727 171.833,71.9293 177.003,77.6248 182.172,83.134 187.342,88.4296 192.511,93.4812 197.681,98.2553 
 202.85,102.715 208.02,106.817 213.19,110.516 218.359,113.759 223.529,116.484 228.698,118.622 233.868,120.102 239.037,120.88 244.207,120.876 249.376,119.967 
 254.546,118.014 259.716,114.846 264.885,110.258 270.055,104.284 275.224,97.6588 280.394,90.6753 285.563,84.5003 290.733,79.5257 295.902,75.551 301.072,72.13 
 306.241,68.5642 311.411,64.6397 316.581,60.408 321.75,56.0187 326.92,51.6364 332.089,47.3071 337.259,43.0834 342.428,39.0248 347.598,35.1975 352.767,31.6762 
 357.937,28.5569 363.107,25.9358 368.276,23.9125 373.446,22.5974 378.615,22.113 383.785,22.5954 388.954,24.1965 394.124,27.0855 399.293,31.452 404.463,37.4874 
 409.633,44.0806 414.802,50.5029 419.972,56.7682 425.141,62.8893 430.311,68.8781 435.48,74.7456 440.65,80.8363 445.819,87.8865 450.989,95.8081 456.158,104.497 
 461.328,113.864 466.498,123.829 471.667,134.32 476.837,145.274 482.006,156.634 487.176,168.35 492.345,180.379 497.515,192.68 502.684,205.221 507.854,217.968 
 513.024,230.887 518.193,243.951 523.363,257.143 528.532,270.443 533.702,283.836 538.871,297.309 544.041,310.851 549.21,324.45 554.38,338.098 559.549,351.788 
 564.719,365.511 
 "/>